# Coherence-Based Functional Connectivity

**Purpose:** Demonstrate coherence-based FC network construction using magnitude-squared coherence (MSC)

**Inputs:** SEEG data from `data/stereoeeg_patients/`

**Outputs:** MSC matrices, network visualizations, comparison plots

**Date:** 2025-12-11

---

## Overview

This notebook demonstrates the coherence-based functional connectivity (FC) pipeline.

**Key differences from correlation-based FC:**
- Uses magnitude-squared coherence (MSC) instead of Pearson correlation
- Captures linear oscillatory coupling (amplitude + phase consistency)
- Employs multitaper spectral estimation for robust frequency-domain analysis
- Optional soft sparsification via circular shift surrogates

**Output format:**
- Same as correlation-based FC: dict of {band_name: adjacency_matrix}
- Compatible with downstream LRG analysis code

In [ ]:
# %% Configuration
CONFIG = {
    'patients': ['Pat_02'],
    'phase': 'rsPre',  # Resting state pre-task
    'bands': 'BRAIN_BANDS',  # Use standard brain bands, or define custom
    
    # MSC computation parameters
    'nperseg': 512,  # Window length for dense MSC
    'nperseg_sparse': 256,  # Window length for sparse MSC (faster)
    
    # Sparsification parameters
    'n_surrogates': 50,  # Number of surrogates (use 200 for final analysis)
    'sparsify': 'soft',  # 'soft', 'none', or other methods
    
    # Visualization
    'compare_band': 'alpha',  # Band for dense vs sparse comparison
    'network_threshold': 0.01,  # Threshold for network statistics
    
    # Output
    'output_dir': 'coherence_fc_tutorial',
}

In [ ]:
# %% Setup
from lrgsglib import move_to_rootf
move_to_rootf(pathname='lrg_eegfc')

from lrg_eegfc.notebook import *
import networkx as nx

# Setup notebook environment
path_figs = setup_notebook(CONFIG['output_dir'])

## 1. Load Data

Load SEEG data using the standard data loader.

In [ ]:
# Load data
data_dict, int_label_map = load_data_dict(pat_list=CONFIG['patients'])
print('✓ Data loaded')
print(f'  Patients: {list(data_dict.keys())}')
print(f'  Phases: {list(data_dict[CONFIG["patients"][0]].keys())}')

## 2. Select Patient and Phase

Extract data for the specified patient and recording phase.

In [ ]:
# Extract patient and phase data
patient_id = CONFIG['patients'][0]
phase = CONFIG['phase']

patient_data = data_dict[patient_id][phase]
X = patient_data['data']  # (N, L) array
fs = patient_data['fs'][0][0] if patient_data['fs'].ndim > 0 else patient_data['fs']  # Sampling frequency

print(f'Patient: {patient_id}')
print(f'Phase: {phase}')
print(f'Data shape: {X.shape} (N channels, L samples)')
print(f'Sampling frequency: {fs} Hz')
print(f'Duration: {X.shape[1] / fs:.2f} seconds')

## 3. Coherence-Based FC Pipeline

### 3.1 Dense MSC (No Sparsification)

Compute band-averaged magnitude-squared coherence without sparsification.

In [ ]:
# Compute dense MSC matrices
msc_matrices = coherence_fc_pipeline(
    X,
    fs,
    bands=BRAIN_BANDS,
    sparsify="none",
    nperseg=CONFIG['nperseg'],
)

print('✓ Dense MSC matrices computed')
print(f'  Bands: {list(msc_matrices.keys())}')
print(f'  Matrix shape: {msc_matrices[list(msc_matrices.keys())[0]].shape}')

### 3.2 Visualize Dense MSC Matrices

In [ ]:
# Plot MSC matrices for all bands
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, (band_name, W) in enumerate(msc_matrices.items()):
    ax = axes[idx]
    im = ax.imshow(W, cmap='hot', vmin=0, vmax=1)
    ax.set_title(f'{band_name.upper()}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Channel')
    ax.set_ylabel('Channel')
    plt.colorbar(im, ax=ax, label='MSC')

plt.tight_layout()
plt.savefig(path_figs / f'{patient_id}_{phase}_msc_dense.png', dpi=150)
plt.show()

print('✓ Dense MSC matrices visualized')

### 3.3 Soft Sparsification with Surrogates

Apply soft sparsification using circular shift surrogates to downweight non-significant edges.

**Note:** This may take a few minutes depending on the number of surrogates and data size.

In [ ]:
# Compute soft-sparsified adjacency matrices
adj_matrices = coherence_fc_pipeline(
    X,
    fs,
    bands=BRAIN_BANDS,
    n_surrogates=CONFIG['n_surrogates'],
    sparsify=CONFIG['sparsify'],
    nperseg=CONFIG['nperseg_sparse'],
    zero_diagonal=True,
)

print('✓ Soft-sparsified adjacency matrices computed')
print(f'  Bands: {list(adj_matrices.keys())}')
print(f'  Matrix shape: {adj_matrices[list(adj_matrices.keys())[0]].shape}')

### 3.4 Visualize Soft-Sparsified Matrices

In [ ]:
# Plot soft-sparsified adjacency matrices
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, (band_name, A) in enumerate(adj_matrices.items()):
    ax = axes[idx]
    im = ax.imshow(A, cmap='hot', vmin=0, vmax=np.max(A))
    ax.set_title(f'{band_name.upper()} (Soft-Sparsified)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Channel')
    ax.set_ylabel('Channel')
    plt.colorbar(im, ax=ax, label='Weighted MSC')

plt.tight_layout()
plt.savefig(path_figs / f'{patient_id}_{phase}_msc_soft_sparsified.png', dpi=150)
plt.show()

print('✓ Soft-sparsified adjacency matrices visualized')

### 3.5 Compare Dense vs Sparsified

Compare weight distributions before and after soft sparsification.

In [ ]:
# Compare for specified band
band = CONFIG['compare_band']

# Extract upper triangular values (excluding diagonal)
N = msc_matrices[band].shape[0]
triu_idx = np.triu_indices(N, k=1)

W_values = msc_matrices[band][triu_idx]
A_values = adj_matrices[band][triu_idx]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Dense MSC histogram
axes[0].hist(W_values, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('MSC Value', fontsize=11)
axes[0].set_ylabel('Count', fontsize=11)
axes[0].set_title(f'{band.upper()} - Dense MSC Distribution', fontsize=12, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].axvline(W_values.mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean={W_values.mean():.3f}')
axes[0].legend()

# Soft-sparsified histogram
axes[1].hist(A_values, bins=50, color='coral', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Weighted MSC Value', fontsize=11)
axes[1].set_ylabel('Count', fontsize=11)
axes[1].set_title(f'{band.upper()} - Soft-Sparsified Distribution', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].axvline(A_values.mean(), color='red', linestyle='--', linewidth=2,
                label=f'Mean={A_values.mean():.3f}')
axes[1].legend()

plt.tight_layout()
plt.savefig(path_figs / f'{patient_id}_{phase}_{band}_comparison.png', dpi=150)
plt.show()

print(f'✓ Comparison for {band} band')
print(f'  Dense MSC - Mean: {W_values.mean():.4f}, Std: {W_values.std():.4f}')
print(f'  Soft-Sparsified - Mean: {A_values.mean():.4f}, Std: {A_values.std():.4f}')
print(f'  Sparsity (< 0.01): {(A_values < 0.01).sum() / len(A_values) * 100:.1f}%')

## 4. Network Statistics

Compute basic network statistics for the soft-sparsified adjacency matrices.

In [ ]:
# Compute network statistics for each band
threshold = CONFIG['network_threshold']
stats = {}

for band_name, A in adj_matrices.items():
    # Create weighted graph
    G = nx.from_numpy_array(A)
    
    # Remove very weak edges
    edges_to_remove = [(u, v) for u, v, w in G.edges(data='weight') if w < threshold]
    G.remove_edges_from(edges_to_remove)
    
    stats[band_name] = {
        'n_nodes': G.number_of_nodes(),
        'n_edges': G.number_of_edges(),
        'density': nx.density(G),
        'n_components': nx.number_connected_components(G),
        'mean_degree': np.mean([d for n, d in G.degree()]),
    }

# Display statistics
print(f'Network Statistics (threshold = {threshold}):') 
print('=' * 70)
for band_name, stat in stats.items():
    print(f'{band_name:12s} | Nodes: {stat["n_nodes"]:3d} | Edges: {stat["n_edges"]:4d} | '
          f'Density: {stat["density"]:.4f} | Components: {stat["n_components"]:2d} | '
          f'Mean Degree: {stat["mean_degree"]:.2f}')
print('=' * 70)

## 5. Custom Frequency Bands (Optional)

Demonstrate using custom frequency bands instead of the standard brain bands.

In [ ]:
# Define custom bands
custom_bands = {
    'slow': (0.5, 4.0),
    'medium': (4.0, 30.0),
    'fast': (30.0, 100.0),
}

# Compute MSC for custom bands
custom_adj = coherence_fc_pipeline(
    X,
    fs,
    bands=custom_bands,
    n_surrogates=CONFIG['n_surrogates'],
    sparsify=CONFIG['sparsify'],
    nperseg=CONFIG['nperseg_sparse'],
)

print('✓ Custom band adjacency matrices computed')
print(f'  Custom bands: {list(custom_adj.keys())}')

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (band_name, A) in enumerate(custom_adj.items()):
    ax = axes[idx]
    im = ax.imshow(A, cmap='hot', vmin=0, vmax=np.max(A))
    fmin, fmax = custom_bands[band_name]
    ax.set_title(f'{band_name.upper()} ({fmin}-{fmax} Hz)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Channel')
    ax.set_ylabel('Channel')
    plt.colorbar(im, ax=ax, label='Weighted MSC')

plt.tight_layout()
plt.savefig(path_figs / f'{patient_id}_{phase}_custom_bands.png', dpi=150)
plt.show()

## 6. Integration with LRG Analysis

The output format is identical to correlation-based FC, so downstream LRG analysis works unchanged.

The adjacency matrices can be directly used with LRG clustering and ultrametric analysis functions.

In [ ]:
print('✓ Adjacency matrices ready for LRG analysis')
print('  ')
print('  Use adj_matrices[band_name] with:')
print('  - plot_lrg_full_panel() for complete LRG analysis')
print('  - plot_lrg_dendrogram() for hierarchical clustering')
print('  - plot_lrg_entropy_curves() for entropy analysis')
print('  - Ultrametric distance comparisons')
print('  ')
print('  Example:')
print('  >>> from lrg_eegfc.visuals import plot_lrg_full_panel')
print('  >>> G = nx.from_numpy_array(adj_matrices["alpha"])')
print('  >>> plot_lrg_full_panel(G, labels=channel_labels, ...)')

## Summary

This notebook demonstrated:

1. **Loading SEEG data** using the standard data loader
2. **Computing dense MSC matrices** without sparsification
3. **Soft sparsification** using circular shift surrogates
4. **Comparing dense vs sparsified** weight distributions
5. **Computing network statistics** from adjacency matrices
6. **Using custom frequency bands** for analysis

### Key Advantages of Coherence-Based FC

- Captures frequency-specific linear oscillatory coupling
- Robust multitaper spectral estimation
- Soft sparsification preserves weight geometry (no hard thresholding)
- Compatible with existing LRG analysis pipeline

### Next Steps

- Apply to all patients and phases
- Use with LRG ultrametric analysis (see tutorial 05)
- Compare correlation-based vs coherence-based FC (see tutorial 04)